# h5 Data Exploration

In this notebook, I will be looking at some data provided to me in the .h5 format, which is traditionally used to store large amounts of data. 

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
import yfinance as yf
import pickle

import h5py
import os

In [3]:
os.listdir('../../')

['SP500_NLP_Project', 'universe_sp500.h5', 'returns_sp500.h5']

In [4]:
f = h5py.File('../../returns_sp500.h5', 'r')

In [5]:
f.keys()

<KeysViewHDF5 ['returns']>

In [6]:
h = f['returns']

In [7]:
h.keys()

<KeysViewHDF5 ['axis0', 'axis1', 'block0_items', 'block0_values']>

In [8]:
dset = h['axis0']
dset.shape

(1110,)

In [9]:
dset[:20]

array([b'A', b'AABA', b'AAL', b'AAMRQ', b'AAP', b'AAPL', b'ABBV', b'ABC',
       b'ABI', b'ABKFQ', b'ABMD', b'ABS', b'ABT', b'ABX', b'ACAS',
       b'ACKH', b'ACN', b'ACS', b'ACV', b'ADBE'], dtype='|S5')

Those appear to be the tickers for the stocks in the S&P 500. Let us continue exploring:

In [10]:
dset2 = h['axis1']
dset2.shape

(6451,)

In [11]:
dset2[:20]

array([820540800000000000, 820627200000000000, 820713600000000000,
       820800000000000000, 821059200000000000, 821145600000000000,
       821232000000000000, 821318400000000000, 821404800000000000,
       821664000000000000, 821750400000000000, 821836800000000000,
       821923200000000000, 822009600000000000, 822268800000000000,
       822355200000000000, 822441600000000000, 822528000000000000,
       822614400000000000, 822873600000000000])

I believe that these numbers represent timestamps in Unix time. 

In [12]:
dset3 = h['block0_items']
dset3.shape

(1110,)

In [13]:
dset3[:20]

array([b'A', b'AABA', b'AAL', b'AAMRQ', b'AAP', b'AAPL', b'ABBV', b'ABC',
       b'ABI', b'ABKFQ', b'ABMD', b'ABS', b'ABT', b'ABX', b'ACAS',
       b'ACKH', b'ACN', b'ACS', b'ACV', b'ADBE'], dtype='|S5')

In [14]:
dset4 = h['block0_values']
dset4.shape

(6451, 1110)

In [15]:
dset4[0]

array([nan, nan, nan, ..., nan, nan, nan])

# Other File

Let's take a look at the other file as well.

In [16]:
f = h5py.File('../../universe_sp500.h5', 'r')

In [17]:
f.keys()

<KeysViewHDF5 ['sp500']>

In [18]:
h = f['sp500']
h.keys()

<KeysViewHDF5 ['axis0', 'axis1', 'block0_items', 'block0_values']>

In [19]:
dset = h['axis0']
dset.shape

(1110,)

In [20]:
dset

<HDF5 dataset "axis0": shape (1110,), type "|S5">

In [21]:
dset[:20]

array([b'A', b'AABA', b'AAL', b'AAMRQ', b'AAP', b'AAPL', b'ABBV', b'ABC',
       b'ABI', b'ABKFQ', b'ABMD', b'ABS', b'ABT', b'ABX', b'ACAS',
       b'ACKH', b'ACN', b'ACS', b'ACV', b'ADBE'], dtype='|S5')

In [22]:
dset2 = h['axis1']
dset2.shape

(9352,)

In [23]:
dset2[:20]

array([820540800000000000, 820627200000000000, 820713600000000000,
       820800000000000000, 820886400000000000, 820972800000000000,
       821059200000000000, 821145600000000000, 821232000000000000,
       821318400000000000, 821404800000000000, 821491200000000000,
       821577600000000000, 821664000000000000, 821750400000000000,
       821836800000000000, 821923200000000000, 822009600000000000,
       822096000000000000, 822182400000000000])

In [24]:
dset3 = h['block0_items']
dset3.shape

(1110,)

In [25]:
dset3[:20]

array([b'A', b'AABA', b'AAL', b'AAMRQ', b'AAP', b'AAPL', b'ABBV', b'ABC',
       b'ABI', b'ABKFQ', b'ABMD', b'ABS', b'ABT', b'ABX', b'ACAS',
       b'ACKH', b'ACN', b'ACS', b'ACV', b'ADBE'], dtype='|S5')

In [26]:
dset4 = h['block0_values']
dset4.shape

(1,)

In [27]:
dset4[:200]

array([array([128,   4, 149, ..., 148,  98,  46], dtype=uint8)],
      dtype=object)

I'm not sure what this final column represents, however it's useful to know that the first dataset contains all the data that we need to calculate returns.

# Construct DataFrame

Now that we know what we have, it will be easy to construct a time series of the stocks and their returns. Let's return to the first dataset and see if we can construct a dataframe.

In [28]:
f = h5py.File('../../returns_sp500.h5', 'r')
h = f['returns']

In [29]:
h.keys()

<KeysViewHDF5 ['axis0', 'axis1', 'block0_items', 'block0_values']>

In [30]:
tickers = np.array(h['axis0'])
tickers

array([b'A', b'AABA', b'AAL', ..., b'ZBRA', b'ZION', b'ZTS'], dtype='|S5')

In [31]:
dates = np.array(h['axis1'])
dates

array([ 820540800000000000,  820627200000000000,  820713600000000000, ...,
       1628035200000000000, 1628121600000000000, 1628208000000000000])

In [32]:
# Verify that it is in fact Unix time
#
# Floor division by nanoseconds because we don't need that much 
# precision to verify
dt = datetime.fromtimestamp(dates[0] // 1000000000)
s = dt.strftime('%Y-%m-%d %H:%M:%S')
s

'1996-01-02 01:00:00'

In [33]:
# I believe these are duplicates of the tickers
tickers2 = np.array(h['block0_items'])
all(tickers2 == tickers)

True

In [34]:
rets = pd.DataFrame(np.array(h['block0_values']), columns = tickers, index=dates)
rets

,b'A',b'AABA',b'AAL',b'AAMRQ',b'AAP',b'AAPL',b'ABBV',b'ABC',b'ABI',b'ABKFQ',...,b'XRX',b'XTO',b'XYL',b'YNR',b'YRCW',b'YUM',b'ZBH',b'ZBRA',b'ZION',b'ZTS'
820540800000000000,NaN,NaN,NaN,NaN,NaN,0.007843,NaN,NaN,NaN,NaN,...,-0.003650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
820627200000000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,...,-0.000915,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
820713600000000000,NaN,NaN,NaN,NaN,NaN,-0.017508,NaN,NaN,NaN,NaN,...,-0.021082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
820800000000000000,NaN,NaN,NaN,NaN,NaN,0.085150,NaN,NaN,NaN,NaN,...,-0.011236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
821059200000000000,NaN,NaN,NaN,NaN,NaN,0.010948,NaN,NaN,NaN,NaN,...,-0.001894,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1627862400000000000,-0.001044,NaN,-0.015702,NaN,0.003065,-0.002331,-0.007309,0.004584,NaN,NaN,...,NaN,NaN,-0.001669,NaN,NaN,0.007611,-0.002509,-0.002733,-0.009971,-0.002911
1627948800000000000,0.005161,NaN,-0.005483,NaN,0.012646,0.012644,0.006929,0.007985,NaN,NaN,...,NaN,NaN,0.018227,NaN,NaN,0.007780,-0.057052,-0.007677,0.021499,0.009846
1628035200000000000,0.002665,NaN,-0.025063,NaN,-0.026602,-0.002782,-0.010065,0.014874,NaN,NaN,...,NaN,NaN,-0.009927,NaN,NaN,0.000750,-0.035652,0.024253,-0.009670,0.003381
1628121600000000000,0.004991,NaN,0.075064,NaN,-0.015882,0.000749,0.000348,-0.029152,NaN,NaN,...,NaN,NaN,0.006790,NaN,NaN,0.000899,0.013088,0.016464,0.017040,-0.004639


In [35]:
rets.columns = [x.decode("utf-8") for x in tickers]
rets

,A,AABA,AAL,AAMRQ,AAP,AAPL,ABBV,ABC,ABI,ABKFQ,...,XRX,XTO,XYL,YNR,YRCW,YUM,ZBH,ZBRA,ZION,ZTS
820540800000000000,NaN,NaN,NaN,NaN,NaN,0.007843,NaN,NaN,NaN,NaN,...,-0.003650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
820627200000000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,...,-0.000915,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
820713600000000000,NaN,NaN,NaN,NaN,NaN,-0.017508,NaN,NaN,NaN,NaN,...,-0.021082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
820800000000000000,NaN,NaN,NaN,NaN,NaN,0.085150,NaN,NaN,NaN,NaN,...,-0.011236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
821059200000000000,NaN,NaN,NaN,NaN,NaN,0.010948,NaN,NaN,NaN,NaN,...,-0.001894,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1627862400000000000,-0.001044,NaN,-0.015702,NaN,0.003065,-0.002331,-0.007309,0.004584,NaN,NaN,...,NaN,NaN,-0.001669,NaN,NaN,0.007611,-0.002509,-0.002733,-0.009971,-0.002911
1627948800000000000,0.005161,NaN,-0.005483,NaN,0.012646,0.012644,0.006929,0.007985,NaN,NaN,...,NaN,NaN,0.018227,NaN,NaN,0.007780,-0.057052,-0.007677,0.021499,0.009846
1628035200000000000,0.002665,NaN,-0.025063,NaN,-0.026602,-0.002782,-0.010065,0.014874,NaN,NaN,...,NaN,NaN,-0.009927,NaN,NaN,0.000750,-0.035652,0.024253,-0.009670,0.003381
1628121600000000000,0.004991,NaN,0.075064,NaN,-0.015882,0.000749,0.000348,-0.029152,NaN,NaN,...,NaN,NaN,0.006790,NaN,NaN,0.000899,0.013088,0.016464,0.017040,-0.004639


In [36]:
rets.index = pd.to_datetime([datetime.fromtimestamp(x // 1000000000).strftime('%Y-%m-%d %H:%M:%S') for x in dates])
rets

,A,AABA,AAL,AAMRQ,AAP,AAPL,ABBV,ABC,ABI,ABKFQ,...,XRX,XTO,XYL,YNR,YRCW,YUM,ZBH,ZBRA,ZION,ZTS
1996-01-02 01:00:00,NaN,NaN,NaN,NaN,NaN,0.007843,NaN,NaN,NaN,NaN,...,-0.003650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-01-03 01:00:00,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,...,-0.000915,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-01-04 01:00:00,NaN,NaN,NaN,NaN,NaN,-0.017508,NaN,NaN,NaN,NaN,...,-0.021082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-01-05 01:00:00,NaN,NaN,NaN,NaN,NaN,0.085150,NaN,NaN,NaN,NaN,...,-0.011236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-01-08 01:00:00,NaN,NaN,NaN,NaN,NaN,0.010948,NaN,NaN,NaN,NaN,...,-0.001894,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-02 02:00:00,-0.001044,NaN,-0.015702,NaN,0.003065,-0.002331,-0.007309,0.004584,NaN,NaN,...,NaN,NaN,-0.001669,NaN,NaN,0.007611,-0.002509,-0.002733,-0.009971,-0.002911
2021-08-03 02:00:00,0.005161,NaN,-0.005483,NaN,0.012646,0.012644,0.006929,0.007985,NaN,NaN,...,NaN,NaN,0.018227,NaN,NaN,0.007780,-0.057052,-0.007677,0.021499,0.009846
2021-08-04 02:00:00,0.002665,NaN,-0.025063,NaN,-0.026602,-0.002782,-0.010065,0.014874,NaN,NaN,...,NaN,NaN,-0.009927,NaN,NaN,0.000750,-0.035652,0.024253,-0.009670,0.003381
2021-08-05 02:00:00,0.004991,NaN,0.075064,NaN,-0.015882,0.000749,0.000348,-0.029152,NaN,NaN,...,NaN,NaN,0.006790,NaN,NaN,0.000899,0.013088,0.016464,0.017040,-0.004639


# Sanity Check

Let's verify that the dates are indexed correctly and do a sanity check to ensure the returns are also aligned correctly. 

In [37]:
sample = rets.loc[:,'AAPL']
sample.head()

1996-01-02 01:00:00    0.007843
1996-01-03 01:00:00    0.000000
1996-01-04 01:00:00   -0.017508
1996-01-05 01:00:00    0.085150
1996-01-08 01:00:00    0.010948
Name: AAPL, dtype: float64

In [38]:
sanity_check = yf.Ticker('AAPL').history(period="max")
sanity_check

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1980-12-12,0.100600,0.101037,0.100600,0.100600,469033600,0.00,0.0
1980-12-15,0.095789,0.095789,0.095352,0.095352,175884800,0.00,0.0
1980-12-16,0.088790,0.088790,0.088353,0.088353,105728000,0.00,0.0
1980-12-17,0.090540,0.090977,0.090540,0.090540,86441600,0.00,0.0
1980-12-18,0.093165,0.093602,0.093165,0.093165,73449600,0.00,0.0
...,...,...,...,...,...,...,...
2021-08-03,145.591857,147.818517,144.962795,147.139542,64786600,0.00,0.0
2021-08-04,147.049692,147.568903,146.061168,146.730164,56368300,0.00,0.0
2021-08-05,146.760114,147.618828,145.951328,146.839996,46397700,0.00,0.0


In [39]:
sanity_check = sanity_check.loc['1996-01-02':,'Close'].pct_change()
sanity_check

Date
1996-01-02         NaN
1996-01-03    0.000000
1996-01-04   -0.017508
1996-01-05    0.085150
1996-01-08    0.010948
                ...   
2021-08-03    0.012644
2021-08-04   -0.002782
2021-08-05    0.000749
2021-08-06   -0.004767
2021-08-09   -0.000342
Name: Close, Length: 6446, dtype: float64

In [40]:
sample

1996-01-02 01:00:00    0.007843
1996-01-03 01:00:00    0.000000
1996-01-04 01:00:00   -0.017508
1996-01-05 01:00:00    0.085150
1996-01-08 01:00:00    0.010948
                         ...   
2021-08-02 02:00:00   -0.002331
2021-08-03 02:00:00    0.012644
2021-08-04 02:00:00   -0.002782
2021-08-05 02:00:00    0.000749
2021-08-06 02:00:00   -0.004767
Name: AAPL, Length: 6451, dtype: float64

Looks good!

# Export the Data

It is important to point out that the times in the df index are not correct, since these returns are calculated based on EOD close. At this point, I believe it is safe to export to a .csv now, but first I will fix this index:

In [41]:
rets.index = pd.to_datetime([datetime.fromtimestamp(x // 1000000000).strftime('%Y-%m-%d') for x in dates])
rets

,A,AABA,AAL,AAMRQ,AAP,AAPL,ABBV,ABC,ABI,ABKFQ,...,XRX,XTO,XYL,YNR,YRCW,YUM,ZBH,ZBRA,ZION,ZTS
1996-01-02,NaN,NaN,NaN,NaN,NaN,0.007843,NaN,NaN,NaN,NaN,...,-0.003650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-01-03,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,...,-0.000915,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-01-04,NaN,NaN,NaN,NaN,NaN,-0.017508,NaN,NaN,NaN,NaN,...,-0.021082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-01-05,NaN,NaN,NaN,NaN,NaN,0.085150,NaN,NaN,NaN,NaN,...,-0.011236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-01-08,NaN,NaN,NaN,NaN,NaN,0.010948,NaN,NaN,NaN,NaN,...,-0.001894,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-02,-0.001044,NaN,-0.015702,NaN,0.003065,-0.002331,-0.007309,0.004584,NaN,NaN,...,NaN,NaN,-0.001669,NaN,NaN,0.007611,-0.002509,-0.002733,-0.009971,-0.002911
2021-08-03,0.005161,NaN,-0.005483,NaN,0.012646,0.012644,0.006929,0.007985,NaN,NaN,...,NaN,NaN,0.018227,NaN,NaN,0.007780,-0.057052,-0.007677,0.021499,0.009846
2021-08-04,0.002665,NaN,-0.025063,NaN,-0.026602,-0.002782,-0.010065,0.014874,NaN,NaN,...,NaN,NaN,-0.009927,NaN,NaN,0.000750,-0.035652,0.024253,-0.009670,0.003381
2021-08-05,0.004991,NaN,0.075064,NaN,-0.015882,0.000749,0.000348,-0.029152,NaN,NaN,...,NaN,NaN,0.006790,NaN,NaN,0.000899,0.013088,0.016464,0.017040,-0.004639


In [42]:
# File too large for github!
#rets.to_csv('1-returns.csv')

In [44]:
rets.to_hdf('1-rets.h5','df',complevel=1,complib='bzip2')